In [22]:
from pydantic import BaseModel
from pathlib import Path
from typing import Optional

class DataSource(BaseModel):
    id: str
    source : str
    type : Optional[str] = None
    link : str

class DataIngestionConfig(BaseModel):
    data_sources : dict[str, DataSource]
    outdir : Path

In [23]:
import os
os.chdir("/Users/morizin/Documents/Code/crash-detection-project")

CONFIG_PATH = 'config/config.yaml'

In [24]:
from typeguard import typechecked
import yaml
from pathlib import Path
from box import ConfigBox
from typing import Any

# @typechecked
def load_yaml(path: Path | str, boxed = True) -> ConfigBox | dict[str | Any]:
    if isinstance(path, str):
        path = Path(path)
        
    config = yaml.safe_load(open(path, "r"))
    if boxed:
        config = ConfigBox(config)
    return config

config = load_yaml(CONFIG_PATH)

In [ ]:

print(data_sources_v)

{'gta-crash': DataSource(id='gta-crash', source='kaggle', type='datasets', link='vision-crash-dataset')}


In [ ]:
class ConfigManager:
    def __init__(self, config_path = CONFIG_PATH):
        self.config = load_yaml(CONFIG_PATH)
        self.artifact_path = Path(self.config.artifact_path)
        self.schema_path = self.config.schema_path

        os.makedirs(self.artifact_path, exist_ok= True)

    def get_data_sources(self) -> dict[str, DataSource] :
        data_sources_v = dict()
        for name, source in self.config.data_sources.items():
            data_sources_v[name] = DataSource(
                id = name,
                source=source.source,
                type = source.type if hasattr(source, "type") else None,
                link = source.link 
            )
        return data_sources_v
    
    def get_data_ingestion_config(self, ) -> DataIngestionConfig:

        outdir = self.artifact_path / "data"
        os.makedirs(outdir, exist_ok= True)

        return DataIngestionConfig(
            data_sources=self.get_data_sources(),
            outdir=outdir
        )

In [ ]:
class DataIngestionPipeline:
    def __init__(self, cfg : DataIngestionConfig):
        self.cfg : DataIngestionConfig = cfg
    
    def __call__(self):
        for source in self.cfg.data_sources:
            print(source)